# Clean Data

***Import libraries***

In [110]:
import pandas as pd
import datetime
import ast
import warnings
warnings.filterwarnings("ignore")

### CSV FILE ###

***Read data file***

In [111]:
trips_data = pd.read_csv("../data/raw/csv/combined_csv.csv", sep=",", low_memory=False)

***Delete the null rows***

In [112]:
trips_data.dropna(how="all", inplace=True)

In [113]:
trips_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4144134 entries, 1 to 8288267
Data columns (total 19 columns):
 #   Column               Dtype  
---  ------               -----  
 0   fecha                object 
 1   idTrip               object 
 2   idBike               float64
 3   fleet                float64
 4   trip_minutes         float64
 5   geolocation_unlock   object 
 6   address_unlock       object 
 7   unlock_date          object 
 8   locktype             object 
 9   unlocktype           object 
 10  geolocation_lock     object 
 11  address_lock         object 
 12  lock_date            object 
 13  station_unlock       object 
 14  dock_unlock          float64
 15  unlock_station_name  object 
 16  station_lock         float64
 17  dock_lock            float64
 18  lock_station_name    object 
dtypes: float64(6), object(13)
memory usage: 632.3+ MB


***Modify column contents***

* Fecha

In [114]:
trips_data.rename(columns={'fecha': 'unlock_date1'}, inplace=True)

* idTrip

In [115]:
trips_data['idTrip'] = trips_data['idTrip'].fillna(trips_data['idTrip']).str.slice(stop=9)
trips_data.rename(columns={'idTrip': 'idDriver'}, inplace=True)

* IdBike

In [116]:
trips_data['idBike'] = trips_data['idBike'].astype(int)

* Fleet

In [117]:
trips_data['fleet'] = trips_data['fleet'].astype(int)

* Geolocation_unlock

In [118]:
trips_data['geolocation_unlock'] = trips_data['geolocation_unlock'].apply(ast.literal_eval)
trips_data['latitude_unlock'] = trips_data['geolocation_unlock'].apply(lambda x: x['coordinates'][1])
trips_data['longitude_unlock'] = trips_data['geolocation_unlock'].apply(lambda x: x['coordinates'][0])

In [119]:
trips_data['latitude_unlock'] = trips_data['latitude_unlock'].astype(str)
trips_data['longitude_unlock'] = trips_data['longitude_unlock'].astype(str)

In [120]:
trips_data.drop(['geolocation_unlock'], axis=1, inplace=True)

* Unlock_date

In [121]:
trips_data['unlock_date'] = trips_data['unlock_date'].str.split('T').str[-1]
trips_data.rename(columns={'unlock_date': 'unlock_hour'}, inplace=True)

In [122]:
trips_data['unlock_hour'] = pd.to_datetime(trips_data['unlock_hour'])

In [123]:
trips_data['unlock_hour'] = trips_data['unlock_hour'].dt.time

In [124]:
trips_data.rename(columns={'unlock_date1': 'unlock_date'}, inplace=True)

* Geolocation_lock

In [125]:
trips_data['geolocation_lock'] = trips_data['geolocation_lock'].apply(ast.literal_eval)
trips_data['latitude_lock'] = trips_data['geolocation_lock'].apply(lambda x: x['coordinates'][1])
trips_data['longitude_lock'] = trips_data['geolocation_lock'].apply(lambda x: x['coordinates'][0])

In [126]:
trips_data.drop(['geolocation_lock'], axis=1, inplace=True)

* Lock_date

In [127]:
trips_data['lock_date'] = pd.to_datetime(trips_data['lock_date'])
trips_data['lock_date1'] = trips_data['lock_date'].dt.date
trips_data['lock_hour'] = trips_data['lock_date'].dt.time

In [128]:
trips_data.drop(['lock_date'], axis=1, inplace=True)

In [129]:
trips_data.rename(columns={'lock_date1': 'lock_date'}, inplace=True)

* Station_unlock

In [130]:
trips_data['station_unlock'] = pd.to_numeric(trips_data['station_unlock'], errors='coerce')
trips_data['station_unlock'] = trips_data['station_unlock'].fillna(0).astype(int)

* Dock_unlock

In [131]:
trips_data['dock_unlock'] = pd.to_numeric(trips_data['dock_unlock'], errors='coerce')
trips_data['dock_unlock'] = trips_data['dock_unlock'].fillna(0).astype(int)

* Station_lock

In [132]:
trips_data['station_lock'] = pd.to_numeric(trips_data['station_lock'], errors='coerce')
trips_data['station_lock'] = trips_data['station_lock'].fillna(0).astype(int)

* Dock_lock

In [133]:
trips_data['dock_lock'] = pd.to_numeric(trips_data['dock_lock'], errors='coerce')
trips_data['dock_lock'] = trips_data['dock_lock'].fillna(0).astype(int)

***Sort the columns***

In [134]:
trips_data.head().T

,1,3,5,7,9
unlock_date,2022-01-01,2022-01-01,2022-01-01,2022-01-01,2022-01-01
idDriver,489978239,06023769T,50856526F,79006741Q,76959007H
idBike,1718,7340,3861,7657,6653
fleet,1,1,1,1,1
trip_minutes,16.28,7.1,0.48,6.62,8.07
address_unlock,NaN,NaN,NaN,NaN,NaN
unlock_hour,00:02:20,00:07:53,00:09:21,00:09:52,00:09:57
locktype,STATION,STATION,STATION,STATION,STATION
unlocktype,STATION,STATION,STATION,STATION,STATION
address_lock,NaN,NaN,NaN,NaN,NaN


In [135]:
trips_data = trips_data.reindex(columns=['idDriver', 'idBike', 'trip_minutes', 'fleet', 'unlock_date', 'unlock_hour', 'latitude_unlock', 'longitude_unlock', 'address_unlock', 'unlocktype', 'station_unlock', 'dock_unlock', 'unlock_station_name','lock_date', 'lock_hour', 'latitude_lock', 'longitude_lock', 'address_lock', 'locktype', 'station_lock', 'dock_lock', 'lock_station_name'])

In [136]:
trips_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4144134 entries, 1 to 8288267
Data columns (total 22 columns):
 #   Column               Dtype  
---  ------               -----  
 0   idDriver             object 
 1   idBike               int32  
 2   trip_minutes         float64
 3   fleet                int32  
 4   unlock_date          object 
 5   unlock_hour          object 
 6   latitude_unlock      object 
 7   longitude_unlock     object 
 8   address_unlock       object 
 9   unlocktype           object 
 10  station_unlock       int32  
 11  dock_unlock          int32  
 12  unlock_station_name  object 
 13  lock_date            object 
 14  lock_hour            object 
 15  latitude_lock        object 
 16  longitude_lock       object 
 17  address_lock         object 
 18  locktype             object 
 19  station_lock         int32  
 20  dock_lock            int32  
 21  lock_station_name    object 
dtypes: float64(1), int32(6), object(15)
memory usage: 632.3+ MB


***Save the clean file***

In [137]:
trips_data.to_csv('../data/processed/clean_data_trips.csv', index=False)

### JSON FILE ###

***Read data file***

In [138]:
data_stations = pd.read_json('../data/raw/json/combined_json.json', lines=True)

***Concat the columns***

In [139]:
stations_df = pd.json_normalize(data_stations['stations'].explode())

In [140]:
replicas = 264
data_stations_rep = pd.concat([data_stations]*replicas)

In [141]:
data_stations_rep = data_stations_rep.reset_index(drop=True)

In [142]:
data_stations_final = pd.concat([data_stations_rep, stations_df], axis=1, join='outer')

***Modify the columns**

* _id

In [143]:
data_stations_final['_id'] = pd.to_datetime(data_stations_final['_id'])
data_stations_final['date_station'] = data_stations_final['_id'].dt.date
data_stations_final['hour_station'] = data_stations_final['_id'].dt.time

In [144]:
data_stations_final['_id'] = data_stations_final['_id'].astype(str)
data_stations_final['_id'] = data_stations_final['_id'].str.slice(-6)

* hour_station

In [145]:
data_stations_final['hour_station'] = data_stations_final['hour_station'].astype(str)
data_stations_final['hour_station'] = data_stations_final['hour_station'].str.slice(stop=8)

* longitude & latitude

In [148]:
data_stations_final['longitude'] = data_stations_final['longitude'].str.slice(stop=9)
data_stations_final['latitude'] = data_stations_final['latitude'].str.slice(stop=9)

***Select the neccessary columns for lock and unlock stations***

In [149]:
data_stations_final_unlock = data_stations_final[['date_station', 'hour_station', 'longitude', 'latitude', 'light', 'total_bases', 'free_bases', 'no_available', 'dock_bikes']]
data_stations_final_unlock.rename(columns={'date_station': 'unlock_date', 'hour_station': 'unlock_hour', 'longitude': 'longitude_unlock', 'latitude': 'latitude_unlock', 'light' : 'light_unlock', 'total_bases' : 'total_bases_unlock', 'free_bases' : 'free_bases_unlock', 'no_available' : 'no_available_unlock', 'dock_bikes' : 'dock_bikes_unlock'}, inplace=True)

In [150]:
data_stations_final_lock = data_stations_final[['date_station', 'hour_station', 'longitude', 'latitude', 'light', 'total_bases', 'free_bases', 'no_available', 'dock_bikes']]
data_stations_final_lock.rename(columns={'date_station': 'lock_date', 'hour_station': 'lock_hour', 'longitude': 'longitude_lock', 'latitude': 'latitude_lock', 'light' : 'light_lock', 'total_bases' : 'total_bases_lock', 'free_bases' : 'free_bases_lock', 'no_available' : 'no_available_lock', 'dock_bikes' : 'dock_bikes_lock'}, inplace=True)

### MERGE DATA

In [151]:
# trips_data = pd.merge(trips_data, data_stations_final_unlock, how='right', on=['unlock_date', 'unlock_hour', 'longitude_unlock', 'latitude_unlock'])

In [152]:
# trips_data = pd.merge(trips_data, data_stations_final_lock, how='right', on=['lock_date', 'lock_hour', 'longitude_lock', 'latitude_lock'])

***Save the clean data***

In [153]:
# trips_data.to_csv('../data/processed/clean_data_trips_stations.csv', index=False)